In [1]:
import gym
import numpy as np
from IPython import display
import matplotlib.pyplot as plt
import pygame
from IPython.display import display as ipydisplay
from collections import deque
import torch.nn as nn
import imageio
import torch.optim as optim

env = gym.make("Pendulum-v1", render_mode='human')

In [2]:
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)


Observation space: Box([-1. -1. -8.], [1. 1. 8.], (3,), float32)
Action space: Box(-2.0, 2.0, (1,), float32)


In [3]:
nb_actions = env.action_space
nb_obs = env.observation_space

In [4]:
class DQNAgent:
    def __init__ (self):
        self.state_dim = 3
        self.action_dim = 9
        self.lr = 0.01
        self.gamma = 0.98
        self.tau = 0.01
        self.epsilon = 1.0
        self.epsilon_min = 0.001
        self.buffer_size = 100000
        self.batch_size = 64
        self.memory = ReplayBuffer(self.buffer_size)

        self.Q = QNetwork(self.state_dim, self.action_dim, self.lr)
        self.Q_target = QNetwork(self.state_dim, self.action_dim, self.lr)
        self.Q_target.load_state_dict(self.Q.state_dict())

In [5]:
class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)
    
    def put(self, transition):
        self.buffer.append(transition)

In [6]:
import torch.nn as nn

In [7]:
class QNetwork(nn.Module):
    def __init__(self, state_dim, action_dim, q_lr):
        super(QNetwork, self).__init__()
        
        self.fc_1 = nn.Linear(state_dim, 64)
        self.fc_2 = nn.Linear(64, 32)
        self.fc_3 = nn.Linear(64, action_dim)
    
        self.lr = q_lr

        self.optimizer = optim.Adam(self.parameters(), lr=self.lr)

    def forward(self, state):
        a = torch.relu(self.l1(state))
        a = torch.relu(self.l2(a))
        return self.l3(a)
    



In [8]:
agent = DQNAgent()

In [11]:
env.reset()
pygame.init()

# Set up the display
prev_screen = env.render()
plt.imshow(prev_screen)
ipydisplay.display(plt.gcf())

for i_episode in range(100001):
    state = env.reset()
    done = False
    while not done:
        action = env.action_space.sample()  # Random action, replace with agent's action
        state, reward, done, info = env.step(action)
        screen = env.render(mode='rgb_array')

        plt.imshow(screen)
        ipydisplay.clear_output(wait=True)
        ipydisplay.display(plt.gcf())

    if done:
        break

ipydisplay.clear_output(wait=True)
env.close()

TypeError: render() got an unexpected keyword argument 'mode'